

---


# **0) Приветствие!**













Здравстуйте, мы команда: 

**ТриП**


1.   Просто
2.   Понятно
3.   По факту


```
# это был наш слоган по которому мы постараемся подвигаться 
# мы так же решили оставлять коментарии в отдельном блоке как сейчас
```


> По составу все просто


*   **Я** -  Говорящие окно с текстом 
*   **Зырянов Дмитрий** - человек который придумал сам код и собрал комнду 
*   **Коломников Максим** - человек который придумал основной дизайн всего
*   **Трепаков Денис** - человек восоздавший приложение для ползователя




Я готов представить вам решение задачи по треку: 

**"ИИ в образовании"**


```
# Данное решение было собранно из многих кусков статьей из интернета
```


Ну так что приступим... 



---






---


# **1) Импорт библиотек**


In [2]:
! wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
! pip install navec
! pip install nltk
! pip install rake-nltk
! pip install pymorphy2[fast]
! pip install -U pymorphy2-dicts-ru



#1
from navec import Navec
import nltk
nltk.download('punkt')
nltk.download("stopwords")
from tqdm import tqdm
from nltk.corpus import stopwords
from string import punctuation
from scipy.special import softmax

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph = pymorphy2.MorphAnalyzer(lang='ru')



#2
import pandas as pd
import numpy as np
import joblib 
import random


--2021-10-16 16:32:45--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.5’

navec_hudlit_v1_12B 100%[===================>]  50.56M  15.8MB/s    in 4.3s    

2021-10-16 16:32:50 (11.8 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.5’ saved [53012480/53012480]



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!




---

# **2) Импорт файлов**

In [3]:
name_data = 'corr_gibrid_5.5.4_'
df_habr = pd.read_csv(f'{name_data}.csv')
russian_stopwords = stopwords.words("russian")


df_habr.head()
df_habr_clean = df_habr.loc[df_habr['categotis'].isin(['животные', 'спорт', 'литература', 'музыка'])]

df_habr_clean

,catalog,categotis,anim,lit,sport,mus,anim2,mus2,sport2,lit2,a_cng_c,a_cng_gl,a_cng_pril,s_cng_c,s_cng_gl,s_cng_pril,m_cng_c,m_cng_gl,m_cng_pril,l_cng_c,l_cng_gl,l_cng_pril
0,славка,животные,славка,романист,элемент,медиатр,Приапулиды,Ля,Open Encyclopedia Project,Эпос,заяц,прыгает,пушистый,тренер,учит,строгий,Бах,сочиняет,красивый,Пушкин,пишет,понятный
1,абелизавр,животные,абелизавр,нон,австралийский,шкатулка,JSTOR,Рондо,Шиллер,Уравнение,обезьяна,лазеет,милая,спортсмен,занимается,чесный,нота,извлекает,красивый,Гоголь,сочинять,понятный
2,абелизаврид,животные,абелизаврид,массовый,автоспорт,гитара,Аристотель,Греческий язык,Фридрих,Энеида,тигр,смотрет,пушистый,футбол,играет,правельный,гитара,дергает,красивый,Достоевский,выдумывать,понятный
3,абидозавр,животные,абидозавр,корпус,рок,шум,Гаптофитовые водоросли,Ассирийцы,Прыжки с шестом,Мемуары,кошка,видит,ловкая,тенис,кидает,спортивный,укулеле,играет,красивый,Толстой,думать,понятный
4,абиссобротула,животные,абиссобротула,текстов,биатлонистка,арфа,Вестхайде,Ниппур,Старофранцузский язык,Парщиков,медведь,кричит,быстрая,плавец,плавает,ленивый,фортепиана,двигает,красивый,Чехов,открывать,понятный
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169,британский язык,музыка,NaN,NaN,NaN,NaN,Кости,Французский язык,Национальная библиотека Франции,Шерер,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6170,карликовый зайчик,музыка,NaN,NaN,NaN,NaN,Верблюдоводство,Нормативный контроль,Английский язык,Вильгельм,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6171,энциклопедический лексика брокгауза и ефрона,музыка,NaN,NaN,NaN,NaN,Лососеобразные,Программная музыка,Зайцев,Марр,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6172,список стереотипов,музыка,NaN,NaN,NaN,NaN,Оленеводство,Риман,Борис Константинович,Николай Яковлевич,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:

test = pd.read_csv('dataset_disclosed.csv')
test.head()


,id,task,category
0,1,"\nИспользуя данные круговой диаграммы, реши за...",спорт
1,2,\nРеши задачу.\nСредняя солёность добавок в ко...,животные
2,3,"""Вы с другом запланировали небольшие велогонки...",спорт
3,4,"""Прочитай текст в учебнике, посвящённый пропор...",литература
4,5,1-й вопрос\n\nЗаполни пропуски (все слова даны...,спорт


In [5]:
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)

    return np.dot(a, b) / np.linalg.norm(a) / np.linalg.norm(b)

animalTexts = [x for x in df_habr['anim']][:1606]
musicTexts = [x for x in df_habr["mus"]][:1606]
sportTexts = [x for x in df_habr["sport"]][:1606]
literatureTexts = [x for x in df_habr["lit"]][:1606]

allTexts = (animalTexts, musicTexts, sportTexts, literatureTexts)

categories = ("животные", "музыка", "спорт", "литература")
allTexts = [
            [[token.strip() for token in nltk.word_tokenize(' '.join(text).lower())] for text in text_category] for text_category in allTexts
]

token_sets = {
    'животные': set(sum(allTexts[0], [])),
    'музыка': set(sum(allTexts[1], [])),
    'спорт': set(sum(allTexts[2], [])),
    'литература': set(sum(allTexts[3], [])),
}

non_exclusive_words = set()
for word in tqdm(set.union(*token_sets.values())):
    count = 0
    for word_set in token_sets.values():
        if word in word_set:
            count += 1

    if count > 1:
        non_exclusive_words.add(word)

exclusive_word_sets = {name: word_set - non_exclusive_words for name, word_set in token_sets.items()}
stopWordsRu = set(stopwords.words("russian"))
punc = set(punctuation + '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~-—')

def token_to_embed_id(word):
    
    if word in stopWordsRu or word in punc:
        return None

    
    if word in navec.vocab:
        return navec.vocab[word]

 
    for category_name, exclusive_word_set in exclusive_word_sets.items():
         if word in exclusive_word_set:
             return navec.vocab[category_name]

    return navec.vocab['<unk>']


def text_to_ids(text: str):
    tokens = []
    embed_ids = []
    for word in nltk.word_tokenize(text):
        word = word.strip().lower()
        embed_id = token_to_embed_id(word)

        if embed_id is not None:
            tokens.append(word)
            embed_ids.append(embed_id)
    return tokens, embed_ids
classes = [
           'животные',
           'спорт',
           'музыка',
           'литература'
]

classes_emb = [
               navec[class_name] for class_name in classes
]

def get_sentence_predictions(sentence: str):
    tokens, embed_ids = text_to_ids(sentence)
    
    embeddings = np.array([navec.pq[embed_id] for embed_id in embed_ids])
    mean_emb = embeddings.sum(axis=0)
    
    results =[
        np.dot(class_emb, mean_emb) for class_emb in classes_emb
    ]

    results = softmax(results)

    return results

def get_word_significancy(sentence: str, class_: int):
    tokens, embed_ids = text_to_ids(sentence)

    embeddings = np.array([navec.pq[embed_id] for embed_id in embed_ids])

    results = [
        (cosine_similarity(token_emb, classes_emb[class_]), token) for token, token_emb in zip(tokens, embeddings)
    ]
    return sorted(results, reverse=True)

100%|██████████| 57/57 [00:00<00:00, 256794.12it/s]




---


# **3) Показ**

Здесь толко малая часть из функция которые я использовал например:



Код ниже это нглядная визуализация как я использовал Embedding слов
```
# еше ниже
{(-0.04316089, 'каое-то'), (0.11974076, 'приложение'), (0.4995613, 'спорте')}
```
каждое слово имеет свой так называемый вес на ту или иную категорию


In [6]:
sen=text_to_ids("Тут каое-то приложение о спорте ")
preds = get_sentence_predictions(' '.join(sen[0]))
word_significancy = set(get_word_significancy(' '.join(sen[0]), np.argmax(preds)))
print(preds)
print(word_significancy)
print(classes[np.argmax(preds)])

[1.0176993e-10 9.9682170e-01 7.0471418e-10 3.1781646e-03]
{(-0.04316089, 'каое-то'), (0.11974076, 'приложение'), (0.4995613, 'спорте')}
спорт


Код ниже это нглядная визуализация как я использовал библиотеку pymorphy2 а именно:

1. Я получаю всю информацию о слове (роде,месте,времени и тд) 
2. Я изменяю уже новое слова по полученным данным 
3. и последнее это уже замена одного слова другим уже измененым(по роду,числу и тд)

In [7]:
p = morph.parse("птицевод")[0].tag.case
print(p)
print(morph.parse("животное")[0].inflect({'ablt'}).word)

print('-*------------')
old_word = "человек"
info = str(morph.parse(old_word)[0].tag).split(',')
print(morph.parse(old_word)[0].tag.cyr_repr)
rec = info[0]
info = info[1:]
info= info + info[1].split()
del info[1] 


nem_word = 'делать'
print(morph.parse(nem_word)[0].tag.cyr_repr)

if rec == morph.parse(word)[0].tag.POS:
  for i in info:
    try:
      word = morph.parse(word)[0].inflect({i}).word
    except:
      pass
else:
  print('не то')
print(word)

nomn
животным
-*------------
СУЩ,од,мр ед,им
ИНФ,несов,перех
не то
``




---

# **4) Основной замысел**

Здесь я отмечу только несколько уловок 

1. Это то что я удаляю очень много не нужных слов и это приводит к тому, то что нахождение ключевых слов намного легче и точно дается имбеденгу
2. Это то что если в приложении есть хотябы одно имя, то я его заменяю на слово которе более сильно относиться к категории, которую задал пользователь
3. Это то что функция calss_class бирет в себя 3 параметра -  это: вопрос, категория(которую задал пользователь) и категория к каторой относится вопрос. 

*Ps*..  Это сделано потому что если категория(которую задал пользователь) == категория(задачи) то смысл менять задачу я не вижу. (Да можно и без этого но я лишний раз перестраховался)

In [94]:
def calss_class(qwes,class_trans,stop_class):

  pun =  punctuation + '!"#$%&()*+,-. /:;<=>?@[\]^_`{|}~-—'

  nonono_pun = qwes.translate(str.maketrans('', '', punctuation + '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~-—'))

  all = [x.lower() for x in pd.read_csv('all_stop.csv')['stop'][:1345]]
  names = [x.lower() for x in pd.read_csv('all_stop.csv')['names']]



  edit_string_as_list = nonono_pun.lower().split()
  final_list = [word for word in edit_string_as_list if word not in all]

  final_list =[x for x in final_list if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
  final_string = ' '.join(final_list)


  p2 = get_sentence_predictions(final_string)
  word_significancy2 = set(get_word_significancy(final_string, np.argmax(p2)))
  nme = ''

  for x in final_list:
    if x in names:
      nme = nme + " " + x.title()
  if nme == '':
    isNameTxt = False
  else:
    isNameTxt = True
  key2 = ''
  if not isNameTxt:
    
    count = 0
    #Так же вот ниже до -- я отлавливаю список ключевых слов
    if len(final_list) > 5 and len(final_list) < 10 :
      while count != 4:
        try:
          if len(max(word_significancy2)[1]) > 2:
            key2 =key2 + max(word_significancy2)[1]+" "
            word_significancy2.remove(max(word_significancy2)) 
            count = count+1
          else:
            word_significancy2.remove(max(word_significancy2)) 
        except:
          break
    elif len(final_list) == 1:
      key2 =final_string
    elif len(final_list) == 2:
      while count != 1:
          try:
            if len(max(word_significancy2)[1]) > 2:
              key2 =key2 + max(word_significancy2)[1]+" "
              word_significancy2.remove(max(word_significancy2)) 
              count = count+1
            else:
              word_significancy2.remove(max(word_significancy2)) 
          except:
            break
    elif len(final_list) == 3:
      while count != 1:
          try:
            if len(max(word_significancy2)[1]) > 2:
              key2 =key2 + max(word_significancy2)[1]+" "
              word_significancy2.remove(max(word_significancy2)) 
              count = count+1
            else:
              word_significancy2.remove(max(word_significancy2)) 
          except:
            break
    else:
      while count != 4:
          try:
            if len(max(word_significancy2)[1]) > 2:
              key2 =key2 + max(word_significancy2)[1]+" "
              word_significancy2.remove(max(word_significancy2)) 
              count = count+1
            else:
              word_significancy2.remove(max(word_significancy2)) 
          except:
            break
  else:
    key2=nme
#------------------

  def slowa_del(sy,pril,prilk,gl,inf,prics,prics_k,deep,new,nammes):
    for i,el in enumerate(key2.split()): 
      old_word = el
      try:
        info = str(morph.parse(old_word)[0].tag).split(',')
        rec = info[0]
        info = info[1:]
        info= info + info[1].split()
        del info[1] 
      except:
        pass
      nem_word = ''
      if not isNameTxt:
        if rec == 'NOUN':
          nem_word = sy[i]
        elif rec == 'ADJF':
          nem_word = pril[i]
        elif rec == 'ADJS':
          nem_word = prilk[i]
        elif rec == 'VERB':
          nem_word = gl[i]  
        elif rec == 'INFN':
          nem_word = inf[i]
        elif rec == 'PRTF':
          nem_word = prics[i]
        elif rec == 'GRND':
          nem_word = deep[i]
        elif rec == 'PRTS':
          nem_word = prics_k[i]
        
        else:
          nem_word = el
      else:
          nem_word = nammes[i]


      word = nem_word
      try:
          if rec == morph.parse(word)[0].tag.POS:
            for i in info:
              try:
                word = morph.parse(word)[0].inflect({i}).word
              except:
                pass
              if class_trans == 'животные':
                new = new.replace(el,word)
              else:
                new = new.replace(el.title(),word.title())
            

          else:
            new = new.replace(el.title(),word.title())
            pass
      except:
        
          for i in info:
            try:
              word = morph.parse(word)[0].inflect({i}).word
            except:
              pass
        
    
    return new

  new = qwes

  if stop_class == class_trans:
      new = qwes
  elif class_trans == 'животные':
      key_sy_a = [random.choice('зверь хищник обезьяна леопард коати кошка детёныш волк гиена тигр павиан сервал собака зверёк кролик вольера зверюга живность грызун сородич поссум агути вольер птица койот гепард барс пантера носуха олень смилодон ягуар бабуин медведь зверинец лемур тварь лисица тигрица оцелот зверьё животина антилопа кабанёнок виверра цератозавр пума крокодил кинкажу кугуар'.split()) for _ in key2.split()]#Сущ
      key_pr_a = [random.choice('животный двуногий четвероногий дикий травоядный бесхвостый бесшёрстный первобытный человечий сумчатый енотовидный шерстистый саблезубый человеческий пугливый пещерный парнокопытный человекообразный копытный звериный кенгуровый длиннохвостый хищный длинношёрстый кошачий плотоядный водоплавающий чепрачный короткохвостый короткошёрстый остромордый длиннорогий длинношеий свирепый длинношёрстный белолобый свиноподобный полудикий яйцекладущий двухголовый зоологический кровожадный пятнистый псовый короткошёрстный стайный обезьяний собакообразный кроликовый всеядный'.split()) for _ in key2.split()]#прил полное
      key_pr_k_a = [random.choice(['мягок','пушист','гладок']) for _ in key2.split()]#прил краткое
      key_gl_a = [random.choice([x for x in df_habr_clean['a_cng_gl'][:11]]) for _ in key2.split()]#глагол
      key_inf_a = [random.choice(['прыгать','плавать','етсь','смотреть','лететь']) for _ in key2.split()]#как глагол но окончание -ть
      key_prics_a = [random.choice(['приручённый одомашненный приручаемый охотящийся учуявший одичавший обитающий похрюкивающий почуявший околевающий спаривающийся выдрессированный мяукавший вспугнувший загрызенный повизгивавший рычащий взбесившийся ласкающийся обезумевший питающийся лазающий хрюкающий дрессируемый обитавший охотившийся водящийся приручивший обнюхивающий бесившийся выкормленный ощенившийся завидевший обнюхивавший бродивший издохший зарычавший заскуливший рычавший лаявший водившийся издыхающий подстреленный убиваемый обезумивший лакомившийся унюхавший кастрированный вспугнутый мяукающий'.split()]) for _ in key2.split()]#причастие 
      key_prics_k_a = [random.choice(['проплыто','проедено','просмотрено','осмотрено']) for _ in key2.split()]#кратное причастие 
      key_deep_a = [random.choice(['присесть','проев','прилечь']) for _ in key2.split()]# деепр

      key_namess_a = key_sy_a

      new = slowa_del(key_sy_a,key_pr_a,key_pr_k_a,key_gl_a,key_inf_a,key_prics_a,key_prics_k_a,key_deep_a,new,key_namess_a)

  elif class_trans == 'литература':
      key_sy_l = [random.choice('литература беллетристика проза классики автор антология писатель сочинение сборник произведение эссе читатель стихи повесть нон-фикшн хрестоматия публицистика поэма литературовед очерк опус стихотворение однотомник романист книга двухтомник писательство заглавие роман новелла классика сочинительство классик переиздание жизнеописание литератор библиография мемуаристика послесловие поэт поэзия рецензия словесность отрывок жанр прозаик беллетрист предисловие трёхтомник эпиграф'.split()) for _ in key2.split()]
      key_pr_l= [random.choice('литературный беллетристический литературоведческий автобиографический многотомный стихотворный двухтомный мемуарный трёхтомный читательский художественный очерковый поэтический публицистический научно-фантастический компилятивный автобиографичный популяризаторский текстологический юмористический прозаический сатирический писательский подстрочный нравоописательный первопечатный жанровый пушкинский переводной романный эпистолярный реалистический приключенческий злободневный памфлетный биографический фельетонный островской повествовательный символистский драматургический балладный новеллистический эсперантский рукописный антологический энциклопедический цитатный эпический силлабический'.split()) for _ in key2.split()]
      key_pr_k_l= [random.choice('литературный беллетристический литературоведческий автобиографический многотомный стихотворный двухтомный мемуарный трёхтомный читательский художественный очерковый поэтический публицистический научно-фантастический компилятивный автобиографичный популяризаторский текстологический юмористический прозаический сатирический писательский подстрочный нравоописательный первопечатный жанровый пушкинский переводной романный эпистолярный реалистический приключенческий злободневный памфлетный биографический фельетонный островской повествовательный символистский драматургический балладный новеллистический эсперантский рукописный антологический энциклопедический цитатный эпический силлабический'.split()) for _ in key2.split()]
      key_gl_l= [random.choice([x for x in df_habr_clean['l_cng_gl'][:11]]) for _ in key2.split()]
      key_inf_l= [random.choice('переиздавать писаться писать читать рецензировать переиздать отрецензировать зачитываться печататься цитировать экранизировать озаглавить напечататься редактировать написать сочинять исписаться перечитывать прочитывать опубликовать переосмыслять публиковать пописывать зарифмовать процитировать почитывать рифмовать читывать исписываться повествовать перепечатывать посвящаться почерпнуть опоэтизировать предпослать штудировать кропать вдохновляться отредактировать дописываться дочитывать перелагать прочесть дописывать заучиваться затверживать поэтизировать переписываться прочитать думаться'.split()) for _ in key2.split()]
      key_prics_l= [random.choice(['переизданный читающий печатавшийся опубликованный печатаемый цитируемый зарифмованный написавший печатающийся прочитанный писавшийся напечатавший прочитавший предпосланный написанный изданный экранизированный писавший аннотированный посвящённый процитированный публикуемый комментированный литографированный сочинявший цитированный публиковавший рецензируемый цензурованный сочиняющий читанный прочтённый вычитанный почерпнутый озаглавленный прокомментированный дочитанный вдохновивший перепечатанный сочиняемый изображаемый опоэтизированный перебелённый цитирующий обессмертивший печатанный излагающий опубликовавший пародируемый поэтизированный'.split()]) for _ in key2.split()]
      key_prics_p_l= [random.choice(['прочитавшх']) for _ in key2.split()]
      key_deep_l= [random.choice(['дописывая']) for _ in key2.split()]

      key_namess_l= [random.choice(['Толстой','Гоголь','Фет','Пушкин','Чехов','Ершов','Тютчев'])for _ in key2.split()]

      new = slowa_del(key_sy_l,key_pr_l,key_pr_k_l,key_gl_l,key_inf_l,key_prics_l,key_prics_p_l,key_deep_l,new,key_namess_l)

  elif class_trans == 'музыка':
      key_sy_m = [random.choice('музыка мелодия блюз музыкант скрипка аккорд саксофон гитара джаз оркестр фортепьяно виолончель аккомпанемент флейта маримба песня электрогитара аккордеон бас-гитара фортепиано аранжировка нота джаз-оркестр цитра квартет клавесин гитарист соната мандолина рок-н-ролл симфония пение арфа оркестрик пианино бэк-вокал рок-ансамбль диксиленд солист кларнет кантри скрипач валторна корнет-а-пистон рояль вокал певец тромбон звучание полутакт'.split()) for _ in key2.split()]
      key_pr_m = [random.choice('музыкальный джазовый фортепьянный лютневый саксофонный симфонический трёхструнный скрипичный клавесинный органный струнный бравурный многострунный клавишный гитарный минорный сладкозвучный теноровый плясовой сольный танцевальный какофонический оркестровый виолончельный мелодический многоголосный вокальный хоровой семиструнный басовый шестиструнный камерный немузыкальный концертный полифонический контральтовый аккордовый сонатный напевный диатонический смычковый песенный лиричный колоратурный эолов филармонический полнозвучный разухабистый мажорный оперный'.split()) for _ in key2.split()]
      key_pr_k_m = [random.choice(['музыкален']) for _ in key2.split()]
      key_gl_m = [random.choice([x for x in df_habr_clean['m_cng_gl'][:11]]) for _ in key2.split()]
      key_inf_m = [random.choice('аккомпанировать наигрывать подпевать солировать петь музицировать фальшивить аранжировать наяривать спеть допевать разучить наиграть дирижировать запеть зазвучать допеть заиграть распеваться сфальшивить пиликать разучивать выпевать танцевать оркестровать концертировать потренькать напеть тренькать транспонировать певать запевать бисировать бренчать подпеть трелить бацать импровизировать дотанцевать отзвучать высвистывать подыгрывать интонировать перепеть напевать подтанцовывать станцевать вторить распевать насвистеть'.split()) for _ in key2.split()]
      key_prics_m = [random.choice(['прочитавший','прочитанная']) for _ in key2.split()]
      key_prics_p_m = [random.choice(['прочитана']) for _ in key2.split()]
      key_deep_m = [random.choice(['прочитав','рассказывая']) for _ in key2.split()]


      key_namess_m = [random.choice(['Бетховен','Бах','Моцарт','Чайковский','Рахманинов','Шуберт']) for _ in key2.split()]


      new = slowa_del(key_sy_m,key_pr_m,key_pr_k_m,key_gl_m,key_inf_m,key_prics_m,key_prics_p_m,key_deep_m,new,key_namess_m)


  elif class_trans == 'спорт':
    key_sy_s = [random.choice('спорт атлетика спортсмен баскетбол хоккей футбол велоспорт гандбол чемпион тренер теннис футболист пятиборье соревнование велогонки волейбол биатлон чемпионат триатлон фристайл регби хоккеист чемпионка теннисист мини-футбол десятиборье бейсбол легкоатлет троеборье спартакиада мотокросс пауэрлифтинг болельщик крикет картинг сноубординг ватерполист еврокубок волейболист футболистка спортшкола кикбоксинг софтбол матч дельтапланеризм бадминтон атлетизм спортсменка культуризм тренировка'.split()) for _ in key2.split()]
    key_pr_s = [random.choice('олимпийский конькобежный легкоатлетический тренерский спортивный хоккейный соревновательный футбольный предсезонный полуфинальный гиревой кубковый чемпионский волейбольный неспортивный баскетбольный беговой тяжелоатлетический физкультурный стайерский полутяжёлый техничный общефизический конноспортивный турниковый отборочный клубный спартаковский игровой гимнастический боксёрский авиамодельный стоклеточный любительский теннисный планёрный тренировочный горнолыжный межзональный спринтерский комбинационный турнирный бейсбольный лыжный зрелищный фигурный марафонский призовой азартный стадионный'.split()) for _ in key2.split()]
    key_pr_k_s = [random.choice(['болюч','быстр','гибок','ловок','вынослив']) for _ in key2.split()]
    key_gl_s = [random.choice([x for x in df_habr_clean['s_cng_gl'][:11]]) for _ in key2.split()]
    key_inf_s = [random.choice(['учить','играть','ходить','бегать','плавть','бить']) for _ in key2.split()]
    key_prics_s = [random.choice('дисквалифицированный тренирующийся разминающийся соревнующийся выигрывавший проигрывающий развивающий засчитанный гоняющий тренирующий гонявший выигрывающий тренировавшийся игравший выигравший проигравший уделяющий состязающийся состязавшийся выигранный увлекавшийся побивший подкачанный оттачивающий тренируемый добившийся накаченный присуждаемый пасующий переигранный накачанный упражняющийся концертирующий нокаутирующий отгадавший блефующий проигрывавший соревновавшийся загоравший побеждавший катающийся обучаемый победивший доигранный подбадривавший отбивающий отдышавшийся догоняющий разыгрывающий угадавший'.split()) for _ in key2.split()]
    key_prics_k_s = [random.choice('разомнулся потренировался выигрывана проиграна'.split()) for _ in key2.split()]
    key_deep_s = [random.choice('проиграть выиграть дисквалифицировать'.split()) for _ in key2.split()]


    key_namess_s = [random.choice(['Роналдо','Лионель Месси','Дзюба','Неймар','Хабиб']) for _ in key2.split()]


    new = slowa_del(key_sy_s,key_pr_s,key_pr_k_s,key_gl_s,key_inf_s,key_prics_s,key_prics_k_s,key_deep_s,new,key_namess_s)



  new = new[0].upper() + new[1:]

  return new


print(calss_class(test.task[99],'литература',test.category[99]))


Вопрос 1
Пушкин любит кататься на велосипеде с подружками. Но ещё больше Пушкин хочет попасть в городскую команду по велокроссу. Поэтому она решила тренироваться всё лето, чтобы в сентябре пройти отбор и попасть в команду.
Пушкин и её папа составили программу тренировок, по которой девочка должна с каждой тренировкой проезжать расстояние на 6% больше, чем в предыдущей тренировке.
Какое расстояние проехала Пушкин на последней тренировке в июне, если во время первой тренировки она проехала 2 км, а всего за месяц провела 10 тренировок?
При вычислениях округляй результаты до целого числа, проводи вычисления для точности в метрах.
Введи ответ без пробелов и единиц измерения. Переведи результат в км и округли до сотых.
На последней тренировке в июне Пушкин проехала на велосипеде … км.




---


# **5) Export**

In [95]:
As = []
Ss =[]
Ls = []
Ms = []

for i in range(len(test.task)):
  As.append(calss_class(test.task[i],'животные',test.category[i]))
  Ss.append(calss_class(test.task[i],'спорт',test.category[i]))
  Ls.append(calss_class(test.task[i],'литература',test.category[i]))
  Ms.append(calss_class(test.task[i],'музыка',test.category[i]))



In [96]:
sample = pd.read_csv('test_finals.csv')
sample['sport'] = Ss
sample['music'] = Ms
sample['literature'] = Ls
sample['animals'] = As
sample.to_csv('Trip_Bot_cng.csv')




#  

---

